# Neresbot 0.1

Modelos:
* Convolução
* Long Short-Term Memory
* SARIMA

In [28]:
import pandas as pd
import requests
import json
from pprint import pprint
import yahooquery as yq
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback

from tqdm import tqdm

In [29]:
# Função de padronização

def padronização(lista):
    máximo = max(lista)
    mínimo = min(lista)
    return list(map(lambda x: (x-mínimo)/(máximo-mínimo) if (máximo-mínimo) != 0 else 0.5,lista))

# Parâmetros Principais

In [30]:
# Baixar base de dados novamente (True) ou usar a preexistente (False)
download = False
# Usar menos dados e menos processamento
Parâmetros_reduzidos = False
# Criar o índice dos dataframes com base na coluna de datas (não recomendável)
indexing = False

Verif_Date = True

In [31]:
# Parâmetros Principais ##

# Tamanho das séries

Per_List = {0:'1d', 1:'5d', 2:'7d', 3:'60d', 4:'1mo', 5:'3mo', 6:'6mo', 7:'1y', 8:'2y', 9:'5y', 10:'10y', 11:'ytd', 12:'max'}
Interv_List = {0:'1m', 1:'2m', 2:'5m', 3:'15m', 4:'30m', 5:'60m', 6:'90m', 7:'1h', 8:'1d', 9:'5d', 10:'1wk', 11:'1mo', 12:'3mo'}

período = 9
período = Per_List[período]
print('período:',período)

período_res = 5
período_res = Per_List[período_res]
print('período_res:',período_res)

# Tamanho de cada período
intervalo = 8
intervalo = Interv_List[intervalo]
print('intervalo:',intervalo)

# Variação identificada nos períodos seguintes
Variação = 0.1
# Períodos em que o aumento pode ocorrer
Prospecção = 10
# Rótulos negativos, para redução de preço na mesma proporção de aumento
Bidirecional = True
# Rótulos categóricos tentam prever em quantos dias a variação ocorre, do contrário, o resultado será 0 (queda), 1 (manuntenção) ou 2 (aumento).
Categórico = True

# Número deperíodos que compõem um array.
defasagem = 20

# Separação entre teste e validação.
# Não mantenha a soma de ambos maior que 0.5
Teste_porcento = 0.3
Valid_porcento = 0.2

# Número máximo de épocas de treinamento
épocas = 25
# Número de épocas que, consecutivamente iguais em termos de acurácia, encerram o treino
paciência = 70
# Se a acurácia desejada é alcançada, o treino é encerrado
Acurácia_desejada = 0.9

período: 5y
período_res: 3mo
intervalo: 1d


In [32]:
# Metaparâmetros reduzidos


if Parâmetros_reduzidos:
    
    # Períodos em que o aumento pode ocorrer
    Prospecção = 5

    # Número deperíodos que compõem um array.
    defasagem = 7

    # Número máximo de épocas de treinamento
    épocas = 10
    # Número de épocas que, consecutivamente iguais em termos de acurácia, encerram o treino
    paciência = 5
    # Se a acurácia desejada é alcançada, o treino é encerrado
    Acurácia_desejada = 0.8

# Download dos Dados

## Testes

In [33]:
try:
    query = yq.Ticker('PETR4.SA', backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
except:
    None
else:
    PETR4 = query.history(period='6mo', interval='1d',adj_timezone=True)
    PETR4 = PETR4.reset_index()
    print(PETR4.columns)
    PETR4['date'] = pd.to_datetime(PETR4['date'])
    PETR4['date'] = PETR4['date'].dt.tz_localize(None)
    PETR4['date'] = PETR4['date'].dt.date
    PETR4 = PETR4.sort_values('date', ascending=True)

try:
    query = yq.Ticker('PETR3.SA', backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
except:
    None
else:
    PETR3 = query.history(period='3mo', interval='1d',adj_timezone=True)
    PETR3 = PETR3.reset_index()
    PETR3['date'] = pd.to_datetime(PETR4['date'])
    PETR3['date'] = PETR3['date'].dt.tz_localize(None)
    PETR3['date'] = PETR3['date'].dt.date
    PETR3 = PETR3.sort_values('date', ascending=True)


try:
    query = yq.Ticker('ELET5.SA', backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
except:
    None
else:
    ELET5 = query.history(period='3mo', interval='1d',adj_timezone=True)
    ELET5 = ELET5.reset_index()
    ELET5['date'] = pd.to_datetime(PETR4['date'])
    ELET5['date'] = ELET5['date'].dt.tz_localize(None)
    ELET5['date'] = ELET5['date'].dt.date
    ELET5 = ELET5.sort_values('date', ascending=True)

Sim_Set = {'PETR4':PETR4,'ELET5':ELET5}

Index(['symbol', 'date', 'open', 'high', 'low', 'close', 'volume', 'adjclose',
       'dividends'],
      dtype='object')


In [34]:
query = yq.Ticker('AALR3.SA', backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)

df_teste = query.history(period='3mo', interval='1d',adj_timezone=True)

df_teste = df_teste.reset_index()

df_teste['date'] = pd.to_datetime(df_teste['date']).dt.date

df_teste

,symbol,date,open,high,low,close,volume,adjclose
0,AALR3.SA,2023-03-28,22.590000,22.680000,22.490000,22.680000,119200.0,22.680000
1,AALR3.SA,2023-03-29,22.650000,22.700001,22.510000,22.559999,117100.0,22.559999
2,AALR3.SA,2023-03-30,22.559999,22.600000,22.500000,22.530001,132200.0,22.530001
3,AALR3.SA,2023-04-02,22.500000,22.570000,22.420000,22.520000,142700.0,22.520000
4,AALR3.SA,2023-04-03,22.510000,22.650000,22.490000,22.540001,137600.0,22.540001
...,...,...,...,...,...,...,...,...
57,AALR3.SA,2023-06-21,23.240000,23.600000,23.240000,23.600000,1068200.0,23.600000
58,AALR3.SA,2023-06-22,23.620001,23.750000,23.490000,23.510000,550300.0,23.510000
59,AALR3.SA,2023-06-25,23.610001,23.610001,23.340000,23.510000,176900.0,23.510000
60,AALR3.SA,2023-06-26,23.610001,23.610001,23.360001,23.500000,159400.0,23.500000


In [35]:
df_teste.to_csv('df teste.csv', index=False)

**Fim dos testes.**

Os símbolos dos tickers são obtidos manualmente do site: https://www.dadosdemercado.com.br/bolsa/acoes

In [36]:
tickers = pd.read_csv("acoes-listadas.csv")

https://yahooquery.dpguthrie.com/

É criado um dicionário que conterá, como chave, o símbolo do ticker e, como valor, o dataframe relacionando medidas e períodos (dias).

Em cada dataframe é adicionado o devido dia da semana, numericamente.

In [37]:
# Os símbolos das empresas brasileiras recebem o sufixo ".SA".

símbolos = tickers['Código'] + '.SA'

In [38]:
# Verificando símbolos inválidos.

print(yq.Ticker(símbolos,
                backoff_factor=1, 
                progress=True, 
                status_forcelist=[404, 429, 500, 502, 503, 504], 
                asynchronous=True
                ).invalid_symbols)

None


Ajuste dos dados:
* Download e armazenamento:
  * Os índices são resetados
  * É verificado se os dados possuem a coluna 'date':
    * É criada a coluna equivalente aos dias semanais
    * Armazenamento no dict dados1
    * Os dados são salvos na pasta 'dados'

In [39]:
from yq_downloader import downloader

In [40]:
#if download:
    #downloader(tickers = símbolos, período = período, report = True, dir = 'dados')
    #downloader(tickers = símbolos, período = período_res, report = True, dir = 'dados_res')

In [41]:
# Função de Download

def hist_download(símbolos,período,intervalo,dir):
    dados1 = {}
    falhas = []
    sem_datas = []
    datas_repetidas = []

    all_dates = []

    for tckr in tqdm(símbolos):
        query = yq.Ticker(tckr, backoff_factor=0.1, retry=3, status_forcelist=[404, 429, 500, 502, 503, 504], asynchronous=True)
        try:
            histórico = query.history(period=período, interval=intervalo,adj_timezone=True)
        except:
            falhas.append(tckr)
            #print(ticker,"falhou.")
        else:
            histórico = histórico.reset_index()
            if 'date' in histórico.columns: # Ajusta os tempos e salva em dados1
                histórico['date'] = pd.to_datetime(histórico['date'])
                histórico['weekday'] = histórico['date'].dt.weekday # Cria dias da semana
                histórico['weekday'] = padronização(histórico['weekday'])
                histórico['yearday'] = histórico['date'].dt.dayofyear # Cria dias da semana
                histórico['yearday'] = padronização(histórico['yearday'])
                histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
                histórico['date'] = histórico['date'].dt.date # Remove horário
                histórico = histórico.sort_values('date', ascending=True) # Organiza por data
                if histórico['date'].duplicated().any():
                    datas_repetidas.append(tckr)
                
                dados1[tckr] = histórico # Salva tudo em dados1

            else:
                #print(ticker,"não contém datas, mas contém:")
                #print(histórico.columns)
                sem_datas.append(tckr)

    print(len(dados1),"tickers salvos com sucesso.")
    print(len(falhas),"tickers apresentaram falha.")
    print(len(sem_datas),"tickers não contém datas.")
    print(len(datas_repetidas),"tickers contém datas repetidas.")

    # Separa as colunas
    Medidas = []
    print('Medidas:')
    for tckr in dados1:
        for col in dados1[tckr].columns:
            if col not in Medidas:
                Medidas.append(col)
                print(f' {col}')

    # Adiciona colunas às tabelas que não as tinha
    for tckr in dados1.keys():
        for col in Medidas:
            if col not in dados1[tckr].columns:
                dados1[tckr][col] = 0

    # Tornando todos os datasets iguais em termos de datas
    all_dates = set()
    for tckr in dados1.keys():
        all_dates.update(set(dados1[tckr]['date']))
    print(len(all_dates),"datas de negociação encontradas.")

    for tckr in dados1.keys():
        missing_dates = all_dates - set(dados1[tckr]['date'])
        missing_data = pd.DataFrame({'date': list(missing_dates)})
        dados1[tckr] = pd.concat([dados1[tckr], missing_data], ignore_index=True)
        # Classifica o dataframe pela coluna 'date' para garantir a ordem correta
        dados1[tckr] = dados1[tckr].sort_values('date').reset_index(drop=True)
        # Preenche cada valor nulo com o número válido anterior
        dados1[tckr] = dados1[tckr].fillna(method = 'ffill')

    # Cria a pasta se ela não existe
    if not os.path.exists(dir):
        os.makedirs(dir)
        
    # Arquiva cada uma das tabelas em .csv
    for tckr, df in dados1.items():
        caminho_arquivo = os.path.join(dir, f'{tckr}.csv')
        df.to_csv(caminho_arquivo, index=False)

    return (falhas, sem_datas, datas_repetidas, Medidas, all_dates)

In [42]:
if download:
    hist_download(símbolos,
                  período,
                  intervalo,
                  'dados')

In [43]:
if download:
    a,b,c,d,c = hist_download(símbolos,período_res,intervalo,'dados_reduzidos')

# Preparação dos Dados

In [44]:
# Carregando dados .csv

dados1 = {}  # Dicionário vazio para armazenar as tabelas
Shapes = []
Rows = []
Columns = []

if Parâmetros_reduzidos:
    print('Dados reduzidos.')
    pasta = 'dados_reduzidos'
else:
    print('Dados completos.')
    pasta = 'dados'

# Percorre todos os arquivos na pasta
for nome_arquivo in os.listdir(pasta):
    if nome_arquivo.endswith('.csv'):  # Verifica se o arquivo é um arquivo CSV
        caminho_arquivo = os.path.join(pasta, nome_arquivo)  # Caminho completo para o arquivo

        histórico = pd.read_csv(caminho_arquivo)
        if 'date' in histórico.columns: # Ajusta os tempos e salva em dadosRe
            histórico['date'] = pd.to_datetime(histórico['date'])
            histórico['date'] = histórico['date'].dt.tz_localize(None) # Remove timezone
            histórico['date'] = histórico['date'].dt.date # Remove horário
            histórico = histórico.sort_values('date') # Organiza por data
        
        dados1[os.path.splitext(nome_arquivo)[0]] = histórico
        Shapes.append(histórico.shape)
        Rows.append(len(histórico.index))
        Columns.append(len(histórico.columns))

# Imprime o dicionário com as tabelas
print(f'{len(dados1)} históricos salvos.')

Rows = pd.Series(Rows).value_counts()
Columns = pd.Series(Columns).value_counts()


Dados completos.


525 históricos salvos.


In [45]:
Rows

1541    466
1239     59
dtype: int64

In [46]:
Columns

12    466
11     59
dtype: int64

In [47]:
# Separando as tabelas por grupos de semelhantes
 # Por datas
 # Por formato

sets_by_date = []

for tckr in dados1.keys():
    hist = dados1[tckr]

    if len(sets_by_date) == 0:
        sets_by_date.append({})
    
    set_index = 0
    Verif = True

    # Todo ticker começa olhando para primeiro set

    while Verif:
        # Se o set para o qual está olhando (set_index) está vazio, cria um 'date' para o set e coloca o ticker atual no set.
        if len(sets_by_date[set_index]) == 0:
            sets_by_date[set_index][tckr] = hist
            sets_by_date[set_index]['hist'] = hist['date']
            sets_by_date[set_index]['shape'] = hist.shape
            Verif = False                                              # Próximo ticker, set 0
        else:
            if (sets_by_date[set_index]['hist'].equals(hist['date'])) and (sets_by_date[set_index]['shape'] == hist.shape):
                sets_by_date[set_index][tckr] = hist
                Verif = False                                          # Próximo ticker, set 0
            else:
                set_index += 1                                         # Próximo set, set 0
                if set_index == len(sets_by_date):
                    sets_by_date.append({})

print(f'{type(sets_by_date)}, Número de sets: {len(sets_by_date)}')
print(f'{type(sets_by_date[0])},{len(sets_by_date[0])}')
print(type(sets_by_date[0]['PETR4.SA']),len(sets_by_date[0]['PETR4.SA']))


<class 'list'>, Número de sets: 2
<class 'dict'>,468
<class 'pandas.core.frame.DataFrame'> 1541


In [48]:
for Set in sets_by_date:
    print(Set['shape'],len(Set))

(1541, 12) 468
(1239, 11) 61


Maiores empresas na bolsa:
* Petrobras: PETR4.SA, PETR3.SA
* Vale: VALE.SA
* Itaú Unibanco: ITUB4.SA, ITUB3.SA, ITSA4.SA
* Ambev: ABEV.SA, 
* Bradesco: BBDC3.SA,  BBDC4.SA
* Weg: WEGE3.SA
* Santander: SANB3.SA, SANB4.SA
* Eletrobrás: ELET5.SA, ELET6.SA, ELET5.SA
* BTG: BPAC3.SA, BPAC5.SA
* Banco do Brasil: BBAS3.SA

In [49]:
princ_tckrs = ['PETR4.SA',
               'PETR3.SA',
               'VALE.SA',
               'ITUB4.SA',
               'ITUB3.SA',
               'ITSA4.SA',
               'ABEV.SA',
               'BBDC3.SA',
               'BBDC4.SA',
               'WEGE3.SA',
               'SANB3.SA',
               'SANB4.SA',
               'ELET5.SA',
               'ELET6.SA',
               'ELET5.SA',
               'BPAC3.SA',
               'BPAC5.SA',
               'BBAS3.SA']


for tckr in princ_tckrs:
    for i in range(len(sets_by_date)):
        if tckr in sets_by_date[i].keys():
            Shape = sets_by_date[i]['shape']
            print(f'{tckr} - set id: {i}; tamanho do set: {len(sets_by_date[i])}, shape: {Shape}')
            #print(' ',list(sets_by_date[i][tckr].columns))

PETR4.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
PETR3.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
ITUB4.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
ITUB3.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
ITSA4.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
BBDC3.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
BBDC4.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
WEGE3.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
SANB3.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
SANB4.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
ELET5.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
ELET6.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
ELET5.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
BPAC3.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
BPAC5.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)
BBAS3.SA - set id: 0; tamanho do set: 468, shape: (1541, 12)


In [50]:
# Verificação dos formatos das tabelas.

for sets in sets_by_date:
    Len = {}
    for tckr in sets.keys():
        if tckr not in ('hist','shape'):
            Shape = str(sets[tckr].shape)
            if Shape not in Len.keys():
                Len[Shape] = 0
            Len[Shape] += 1
    print(Len)

{'(1541, 12)': 466}
{'(1239, 11)': 59}


Se os ticker existentes em determinados dados não puderemser baixados para o período necessário para a prospecção, não devem ser incluídos no modelo.

In [51]:
dados2 = sets_by_date[0]
if 'hist' in dados2.keys():
    del dados2['hist']
if 'shape' in dados2.keys():
    del dados2['shape']


In [52]:
# Verificando se todas as colunas 'date' são iguais 

if Verif_Date:
    ticker_divergentes = pd.DataFrame(columns=dados2.keys(), index=dados2.keys())

    for column in dados2.keys():
        for index in dados2.keys():
            if dados1[column]['date'].equals(dados2[index]['date']):
                ticker_divergentes.at[index,column] = True
            else:
                ticker_divergentes.at[index,column] = False

    ticker_divergentes.to_csv('ticker_divergentes.csv')
    ticker_divergentes

# Convolução

## Rótulos

Cada linha analisa se há aumento expressivo nos próximos 5 períodos.

Se não houver, o rótulo será 0; Caso contrário, será o número de dias em que o aumento ocorre.  
Há uma variável ajustável para determinar o nível do aumento.

In [53]:
print('Parâmetros_reduzidos:',Parâmetros_reduzidos)
print('Prospecção:',Prospecção)
print('Bidirecional:',Bidirecional)
print('Categórico:',Categórico)

Parâmetros_reduzidos: False
Prospecção: 10
Bidirecional: True
Categórico: True


In [54]:
# 2 - Valores dos Rótulos, separando as buscaspositivas e negativas
Verbose = False
print(f'') if Verbose else None

print('SE BIDIRECIONAL: OS VALORES MAIS ALTOS SERÃO O DOBRO DA PROSPECÇÃO.')
# Coluna para os rótulos

for i,j in dados2.items():
    j['rótulos'] = 0
    j['compar'] = 0

# Valores para os rótulos

for tckr, df in tqdm(dados2.items()):                  # Para cada ticker, tabela:                      # reseta o índice da tabela
    print() if Verbose else None
    print('Ticker:',tckr) if Verbose else None
    
    for i, row in df.iterrows():
        price_now = row['close']
        price_contester_high = row['close']
        price_contester_high_id = i
        price_contester_low = row['close']
        price_contester_low_id = i
        print(' i:',i,'; price_now e contests:',price_now) if Verbose else None


        # Procurando positivos.
        print(' Procurando aumentos:') if Verbose else None
        for j in range(i+1, i+1+Prospecção):
            
            # Se J ainda é dentro da tabela
            if j < len(df):
                print(f'  j:{j}') if Verbose else None
                j_high = df.loc[j, 'high']
                print(f'   high de j={j}: {j_high}.') if Verbose else None
                
                # se o preço avaliado é maior ou igual ao aumento esperado:
                if (j_high >= (1 + Variação)*price_now) and (j_high > price_contester_high):
                    print(f'     j_high ({j_high}) supera a variação esperada ({(1 + Variação)*price_now}) e o último maior ({price_contester_high}).') if Verbose else None
                    if Categórico:
                        dados2[tckr].loc[i, 'rótulos'] = j - i
                    else:
                        dados2[tckr].loc[i, 'rótulos'] = 1
                    dados2[tckr].loc[i, 'compar'] = j_high
                    price_contester_high = j_high
                    price_contester_high_id = j
                    temp = dados2[tckr].loc[i, 'rótulos']
                    print(f'     Rótulo atualizado: {temp}') if Verbose else None
                    print(f'     O último maior preço para o id {i} agora é {price_contester_high}, do id {j}.') if Verbose else None
                else:
                    temp = dados2[tckr].loc[i, 'rótulos']
                    print(f'     Rótulo permanece: {temp}') if Verbose else None
            else:
                print('   j ultrapassa o limite da tabela') if Verbose else None


        
        # Iteração entre os próximos "Prospecção" itens, procurando negativos.
        if Bidirecional:
            print(' Procurando aumentos:') if Verbose else None
            for j in range(i+1, i+1+Prospecção):
                
                # Se J ainda é dentro da tabela
                if j < len(df):
                    print(f'  j:{j}') if Verbose else None
                    j_low = df.loc[j, 'low']
                    print(f'   low de j={j}: {j_low}.') if Verbose else None

                    # se o preço avaliado é menor ou igual à redução esperada e é bidirecional:
                    if (j_low <= (1-Variação)*price_now) and (j_low < price_contester_low) and Bidirecional:
                        print(f'     post_low ({j_low}) supera a variação esperada ({(1 + Variação)*price_now}) e o último maior ({price_contester_low}).') if Verbose else None
                        if Categórico:
                            dados2[tckr].loc[i, 'rótulos'] = i - j
                        else:
                            dados2[tckr].loc[i, 'rótulos'] = -1
                        dados2[tckr].loc[i, 'compar'] = j_low
                        price_contester_low = j_low
                        price_contester_low_id = j
                        temp = dados2[tckr].loc[i, 'rótulos']
                        print(f'     Rótulo atualizado: {temp}') if Verbose else None
                        print(f'     O último menor preço para o id {i} agora é {price_contester_low}, do id {j}.') if Verbose else None
                    else:
                        temp = dados2[tckr].loc[i, 'rótulos']
                        print(f'     Rótulo permanece: {temp}') if Verbose else None
                else:
                    print('   j ultrapassa o limite da tabela') if Verbose else None
                
                

        if dados2[tckr].loc[i, 'rótulos'] == 0:
            print(f'  Nenhum id entre {i+1} e {j} apresentou valores consideráveis') if Verbose else None
            print(f'    j_high depois: {j_high}') if Verbose else None
            print(f'    j_low depois: {j_low}') if Verbose else None


        # Remoção de rótulos negativos
        if Bidirecional:
            if Categórico:
                dados2[tckr].loc[i, 'rótulos'] += Prospecção
            else:
                dados2[tckr].loc[i, 'rótulos'] += 1
        temp = dados2[tckr].loc[i, 'rótulos']
        print(f'    Para o id ({i}) o rótulo definitivo é {temp}') if Verbose else None

        if Verbose:
            if (((dados2[tckr].loc[i, 'rótulos'] != Prospecção) and (Categórico)) or (dados2[tckr].loc[i, 'rótulos'] != 1)) and (tckr == 'ITSA4.SA'):
                print(row['symbol'], dados2[tckr].loc[i, 'rótulos'], (((j_high/price_now)-1)*100), (((j_high/price_now)-1)*100)) if Verbose else None
                print(row['symbol'], dados2[tckr].loc[i, 'rótulos'], (((j_low/price_now)-1)*100), (((j_low/price_now)-1)*100)) if Verbose else None
    
    # print(df.head()) #??

SE BIDIRECIONAL: OS VALORES MAIS ALTOS SERÃO O DOBRO DA PROSPECÇÃO.


100%|██████████| 466/466 [09:49<00:00,  1.26s/it]


In [55]:
print('  Rótulos')
for tckr in dados2:
    print(tckr+':', dados2[tckr]['rótulos'].min(), dados2[tckr]['rótulos'].max())

  Rótulos
AALR3.SA: 0 20
ABCB4.SA: 0 20
ABEV3.SA: 0 20
AERI3.SA: 0 20
AESB3.SA: 0 20
AFLT3.SA: 0 20
AGRO3.SA: 0 20
AGXY3.SA: 0 20
AHEB3.SA: 0 20
AHEB5.SA: 0 20
AHEB6.SA: 0 20
ALLD3.SA: 0 20
ALPA3.SA: 0 20
ALPA4.SA: 0 20
ALPK3.SA: 0 20
ALSO3.SA: 0 20
ALUP3.SA: 0 20
ALUP4.SA: 0 20
AMAR3.SA: 0 20
AMBP3.SA: 0 20
AMER3.SA: 0 20
ANIM3.SA: 0 20
APER3.SA: 0 20
ARML3.SA: 0 20
ARZZ3.SA: 0 20
ASAI3.SA: 0 20
ATMP3.SA: 0 20
ATOM3.SA: 0 20
AVLL3.SA: 0 20
AZEV3.SA: 0 20
AZEV4.SA: 0 20
AZUL4.SA: 0 20
B3SA3.SA: 0 20
BAHI3.SA: 0 20
BALM3.SA: 0 20
BALM4.SA: 0 20
BAUH4.SA: 0 20
BAZA3.SA: 0 20
BBAS3.SA: 0 20
BBDC3.SA: 0 20
BBDC4.SA: 0 20
BBSE3.SA: 0 20
BDLL3.SA: 0 20
BDLL4.SA: 0 20
BEEF3.SA: 0 20
BEES3.SA: 0 20
BEES4.SA: 0 20
BGIP3.SA: 0 20
BGIP4.SA: 0 20
BIOM3.SA: 0 20
BLAU3.SA: 0 20
BMEB3.SA: 0 20
BMEB4.SA: 0 20
BMGB4.SA: 0 20
BMIN3.SA: 0 20
BMIN4.SA: 0 20
BMKS3.SA: 0 20
BMOB3.SA: 0 20
BNBR3.SA: 0 20
BOAS3.SA: 0 20
BOBR4.SA: 0 20
BPAC3.SA: 0 20
BPAC5.SA: 0 20
BPAN4.SA: 0 20
BRAP3.SA: 0 20
BRAP4.SA: 0 20


In [56]:
for tckr in dados2:
    print(dados2[tckr].tail(4))

        symbol        date       open       high        low  close    volume  \
1537  AALR3.SA  2023-06-25  23.610001  23.610001  23.340000  23.51  176900.0   
1538  AALR3.SA  2023-06-26  23.610001  23.610001  23.360001  23.50  159400.0   
1539  AALR3.SA  2023-06-27  23.610001  23.610001  23.360001  23.50  159400.0   
1540  AALR3.SA  2023-06-29  23.680000  23.719999  23.580000  23.65  179000.0   

      adjclose  dividends  weekday   yearday  splits  rótulos  compar  
1537     23.51        0.0      1.0  0.482094     0.0       10     0.0  
1538     23.50        0.0      0.0  0.484848     0.0       10     0.0  
1539     23.50        0.0      0.0  0.484848     0.0       10     0.0  
1540     23.65        0.0      0.5  0.493113     0.0       10     0.0  
        symbol        date       open       high        low      close  \
1537  ABCB4.SA  2023-06-25  18.510000  18.530001  18.040001  18.370001   
1538  ABCB4.SA  2023-06-26  18.799999  19.000000  18.360001  18.620001   
1539  ABCB4.SA  2

In [57]:
print('  Dividendos')
for tckr in dados2:
    print(tckr+':',dados2[tckr]['dividends'].max())

  Dividendos
AALR3.SA: 0.087297
ABCB4.SA: 0.554453
ABEV3.SA: 0.7623
AERI3.SA: 0.020702
AESB3.SA: 0.165705
AFLT3.SA: 0.29471
AGRO3.SA: 3.238673
AGXY3.SA: 0.167864
AHEB3.SA: 0.0
AHEB5.SA: 0.0
AHEB6.SA: 0.0
ALLD3.SA: 0.570983
ALPA3.SA: 0.247202
ALPA4.SA: 0.271712
ALPK3.SA: 0.0
ALSO3.SA: 0.519014
ALUP3.SA: 0.48
ALUP4.SA: 0.48
AMAR3.SA: 0.0
AMBP3.SA: 0.303171
AMER3.SA: 0.620963
ANIM3.SA: 0.0023
APER3.SA: 0.323684
ARML3.SA: 0.118727
ARZZ3.SA: 0.831022
ASAI3.SA: 0.125038
ATMP3.SA: 0.0
ATOM3.SA: 0.320385
AVLL3.SA: 0.0
AZEV3.SA: 0.0
AZEV4.SA: 0.0
AZUL4.SA: 0.0
B3SA3.SA: 0.265538
BAHI3.SA: 0.0
BALM3.SA: 0.34
BALM4.SA: 0.41
BAUH4.SA: 1.20933
BAZA3.SA: 6.809764
BBAS3.SA: 0.826881
BBDC3.SA: 0.712738
BBDC4.SA: 0.784011
BBSE3.SA: 1.910841
BDLL3.SA: 0.0
BDLL4.SA: 0.0
BEEF3.SA: 0.729812
BEES3.SA: 0.181503
BEES4.SA: 0.181503
BGIP3.SA: 2.243082
BGIP4.SA: 2.46739
BIOM3.SA: 0.0
BLAU3.SA: 0.205591
BMEB3.SA: 0.623826
BMEB4.SA: 0.686208
BMGB4.SA: 0.31551
BMIN3.SA: 1.420175
BMIN4.SA: 1.589625
BMKS3.SA: 57.7863

## Transformação das matrizes: Separando os dias como objetos numpy.

Classes que contenham as matrizes, os rótulos e a identificação do dia.

In [58]:
# Todos os dataframes:

df_concatenado = pd.concat(dados2.values(), axis=1)

In [59]:
df_concatenado.to_csv('df_concatenado.csv',index=True)

O último dia não deve estar incluso no treinamento por não possuir rótulo.

Dimensões:
* Medidas:
  * open
  * high
  * low
  * close
  * volume
  * adjclose
  * weekday
  * yearday
  * dividends
  * splits
* tickers
* Dias (atual e anteriores)

In [60]:
# Adição de colunas "necessárias"
# Remoção das colunas "desnecessárias"

Colunas_Necessárias = ['open',
                    'high',
                    'low', 
                    'close', 
                    'volume', 
                    'adjclose', 
                    'weekday', 
                    'yearday', 
                    'dividends', 
                    'splits', 
                    'rótulos']

dados3 = {}

for tckr in dados2.keys():
    for c in Colunas_Necessárias:
        if c not in dados2[tckr].columns:
            dados2[tckr][c] = 0

for tckr in dados2.keys():
    dados3[tckr] = dados2[tckr][Colunas_Necessárias]

In [61]:
dados3['AALR3.SA'].columns

Index(['open', 'high', 'low', 'close', 'volume', 'adjclose', 'weekday',
       'yearday', 'dividends', 'splits', 'rótulos'],
      dtype='object')

In [62]:
# Tendo certeza de que todas as tabelas têm o mesmo tamanho.

for t in dados3.keys():
    for k in dados3.keys():
        Série = len(dados3[t])
        dif = len(dados3[t])-len(dados3[k])
        if dif != 0:
            print(t,'-',k,'=',dif)

In [63]:
Série

1541

Há uma variável ajustável para determinar quantos dias serão olhados para trás.

In [64]:
# Criação lista completa

Tensor = []

Rótulos = {}
for tckr in dados3.keys():
    Rótulos[tckr] = []

for i in range(Série-Prospecção):
    lvl1 = []
    for tckr in dados3.keys():
        lvl2 = []
        for coluna in dados3[tckr].columns:
            if coluna != 'rótulos':
                lvl3 = []
                for j in range(i - defasagem, i):
                    if j >= 0:
                        k = j
                    else:
                        k = 0
                    lvl3.append(dados3[tckr].at[k,coluna])
                    if coluna not in ['weekday','yearday']:
                        lvl3p = padronização(lvl3)
            lvl2.append(lvl3p)
        lvl1.append(lvl2)
        Rótulos[tckr].append(dados3[tckr].at[i,'rótulos'])
    Tensor.append(lvl1)

## Separação entre Treino, Teste e Validação.

É preciso garantir que as matrizes sejam embaralhadas, mas que os rótulos permaneçam atrelados adequadamente.

In [65]:
type(Tensor)

list

In [66]:
Treino_tensor = []
Teste_tensor = []
Valid_tensor = []

Treino_rótulo = {}
for tckr in dados3.keys():
    Treino_rótulo[tckr] = []

Teste_rótulo = {}
for tckr in dados3.keys():
    Teste_rótulo[tckr] = []

Valid_rótulo = {}
for tckr in dados3.keys():
    Valid_rótulo[tckr] = []

id_teste_valid = random.sample(list(range(len(Tensor))), int(round(len(Tensor)*(Teste_porcento+Valid_porcento), 0)))

id_valid = random.sample(id_teste_valid, int(round(len(Tensor)*Valid_porcento, 0)))

for i in range(len(Tensor)):
    if i in id_valid:
        Valid_tensor.append(Tensor[i])
        for tckr in Valid_rótulo.keys():
            Valid_rótulo[tckr].append(Rótulos[tckr][i])
            
    elif i in id_teste_valid:
        Teste_tensor.append(Tensor[i])
        for tckr in Teste_rótulo.keys():
            Teste_rótulo[tckr].append(Rótulos[tckr][i])

    else:
        Treino_tensor.append(Tensor[i])
        for tckr in Treino_rótulo.keys():
            Treino_rótulo[tckr].append(Rótulos[tckr][i])

Treino_tensor = np.array(Treino_tensor)
Teste_tensor = np.array(Teste_tensor)
Valid_tensor = np.array(Valid_tensor)

for tckr in Treino_rótulo.keys():
    Treino_rótulo[tckr] = np.array(Treino_rótulo[tckr])
for tckr in Teste_rótulo.keys():
    Teste_rótulo[tckr] = np.array(Teste_rótulo[tckr])
for tckr in Valid_rótulo.keys():
    Valid_rótulo[tckr] = np.array(Valid_rótulo[tckr])

In [67]:
print(Treino_tensor.shape)
print(Teste_tensor.shape)
print(Valid_tensor.shape)

print()

print(len(Treino_rótulo),len(Treino_rótulo['PETR4.SA']))
print(len(Teste_rótulo),len(Teste_rótulo['PETR4.SA']))
print(len(Valid_rótulo),len(Valid_rótulo['PETR4.SA']))

(765, 466, 11, 20)
(460, 466, 11, 20)
(306, 466, 11, 20)

466 765
466 460
466 306


## Modelo

* Para cada ticker (a partir do dicionário de rótulos):  
    * O modelo é treinado seguindo aquele ticker como rótulo (com uma função)  
    * O modelo é validado  
    * Se o modelo é válido:  
        * A previsão é tentada:  
        * Se o resultado é diferente de zero E a probabilidade é alta:  
                * O deploy é salvo

Exemplo com um ticker: PTR4.SA

In [74]:
Shape = Treino_tensor.shape[1:]
Ativação = 'relu'

print(Shape)

(466, 11, 20)


In [75]:
# Cria o objeto de sequência de camadas
modelo = models.Sequential()

modelo.add(layers.Conv2D(64, (4, 2), activation = 'relu', input_shape = Shape))
modelo.add(layers.MaxPooling2D((2, 2)))

modelo.add(layers.Conv2D(64, (4, 2), activation = 'relu'))
modelo.add(layers.MaxPooling2D((2, 2)))

modelo.add(layers.Conv2D(64, (1, 1), activation = 'relu'))
modelo.add(layers.MaxPooling2D((2, 2)))

modelo.add(layers.Flatten())

modelo.add(layers.Dense(64, activation = 'relu'))

modelo.add(layers.Dense(32, activation = 'relu'))

modelo.add(layers.Dense((Prospecção+1), activation = 'softmax'))

modelo.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 463, 10, 64)       10304     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 231, 5, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 228, 4, 64)        32832     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 114, 2, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 114, 2, 64)        4160      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 57, 1, 64)       

In [76]:
modelo.compile(optimizer = 'adam', 
                   loss = 'sparse_categorical_crossentropy', 
                   metrics = ['accuracy'])

In [77]:
class AccuracyThresholdCallback(Callback):
    def __init__(self, accuracy_threshold):
        super(AccuracyThresholdCallback, self).__init__()
        self.accuracy_threshold = accuracy_threshold

    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= self.accuracy_threshold:
            self.model.stop_training = True
            print(f"\nAtingiu a acurácia desejada ({self.accuracy_threshold}). Treinamento interrompido.")

In [78]:
# Criando um parâmetro de parada com o objeto EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', mode='max', patience=paciência, verbose=1)

# Salvando a melhor época.
checkpoint = ModelCheckpoint('Modelos\\melhor_CNN.h5', monitor='val_accuracy', mode='max', save_best_only=True)

# Criar o objeto do callback personalizado
accuracy_callback = AccuracyThresholdCallback(Acurácia_desejada)

In [73]:
# Treinamento e teste

history = modelo.fit(Treino_tensor, 
                         Treino_rótulo['PETR4.SA'], 
                         epochs = 5, 
                         validation_data = (Teste_tensor, Teste_rótulo['PETR4.SA']),
                         callbacks=[early_stopping, checkpoint])

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\fnsb\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\fnsb\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\fnsb\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\fnsb\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\fnsb\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\Users\fnsb\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\fnsb\anaconda3\lib\a

In [79]:
modelo.evaluate(Valid_tensor, Valid_rótulo['PETR4.SA'], verbose = 2)

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\fnsb\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\fnsb\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\fnsb\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\fnsb\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\fnsb\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\Users\fnsb\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\fnsb\anaconda3\lib\a

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = modelo.evaluate(Valid_tensor, Valid_rótulo['PETR4.SA'], verbose=2)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

test_predictions = modelo.predict(Valid_tensor)

test_predicted_labels = np.argmax(test_predictions, axis=1)
print(test_predicted_labels)

test_true_labels = np.array(Valid_rótulo['PETR4.SA'])
print(test_true_labels)

cm = confusion_matrix(test_true_labels, test_predicted_labels)

cmd = ConfusionMatrixDisplay(confusion_matrix=cm)

cmd.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')
plt.title('PETR4.SA')
plt.show()

In [81]:
if not os.path.exists('conv_confusion_matrix'):
    os.makedirs('conv_confusion_matrix')

shape_saída = (Prospecção*2)+1 if Categórico and Bidirecional else \
              Prospecção+1 if Categórico else \
              3 if Bidirecional else \
              2


Modelos = {}

for tckr in tqdm(Rótulos.keys()):
    # Cria o objeto de sequência de camadas
    
    Modelos[tckr] = models.Sequential()

    Modelos[tckr].add(layers.Conv2D(64, (2, 2), activation = 'relu', input_shape = Shape))
    Modelos[tckr].add(layers.MaxPooling2D((2, 2)))

    Modelos[tckr].add(layers.Conv2D(64, (2, 2), activation = 'relu'))
    Modelos[tckr].add(layers.MaxPooling2D((2, 2)))

    Modelos[tckr].add(layers.Conv2D(64, (1, 1), activation = 'relu'))
    Modelos[tckr].add(layers.MaxPooling2D((2, 2)))

    Modelos[tckr].add(layers.Flatten())

    Modelos[tckr].add(layers.Dense(64, activation = 'relu'))

    Modelos[tckr].add(layers.Dense(32, activation = 'relu'))

    Modelos[tckr].add(layers.Dense(shape_saída, activation = 'softmax'))

    Modelos[tckr].compile(optimizer = 'adam', 
                    loss = 'sparse_categorical_crossentropy', 
                    metrics = ['accuracy'])

    # Treinamento e teste

    history = Modelos[tckr].fit(Treino_tensor, 
                            Treino_rótulo[tckr], 
                            epochs = épocas, 
                            validation_data = (Teste_tensor, Teste_rótulo[tckr]),
                            callbacks = [early_stopping, checkpoint], # accuracy_callback
                            verbose = 0)
    
    Modelos[tckr].evaluate(Valid_tensor, Valid_rótulo[tckr], verbose = 0)
    
    test_predictions = Modelos[tckr].predict(Valid_tensor)

    test_predicted_labels = np.argmax(test_predictions, axis=1)

    test_true_labels = np.array(Valid_rótulo[tckr])

    cm = confusion_matrix(test_true_labels, test_predicted_labels)

    cmd = ConfusionMatrixDisplay(confusion_matrix=cm)

    cmd.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')

    plt.title(tckr+' ('+str(len(test_true_labels))+')')

    plt.savefig('conv_confusion_matrix\\' + tckr+'.png')

  0%|          | 0/466 [00:00<?, ?it/s]

* Objeto de tracking para averiguar se os dados não foram desalinhados
* Salvar a acertividade dos modelos por ticker
* Verificar porque alguns gráficos de calor têm menos observações.